In [ ]:
# %% [markdown]
# # PIAS-VAE End-to-End Pipeline
# This notebook runs:
# 1) Train LSTM-VAE on NORMAL sequences
# 2) Create synthetic anomalies using interpolation + SMOTE-style latent perturbation
# 3) Train LSTM classifier on augmented dataset (best model saved by anomaly F1)
# 4) Threshold calibration for anomaly detection
# 5) Evaluation and artifact saving
#
# Requirements:
# - torch, numpy, pandas, sklearn, matplotlib, seaborn
# - The file `src/vae_time_series.py` (the LSTM VAE) should be present (we used your provided version)
# - This notebook will generate mock CMAPSS-like data if real data is not present
#
# Tweak constants below as needed.

# %%
# Imports & constants
import os
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support, confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Make sure your src module is importable
import sys
sys.path.append('src')

# Import VAE helpers from your src module
from vae_time_series import PIASVAE, train_vae_from_dataloader, synthesize_and_save, \
    encode_dataset_to_latents, decode_latents_to_sequences, SEQUENCE_LENGTH, NUM_FEATURES, LATENT_DIM

# Directory layout
os.makedirs('data/01_raw', exist_ok=True)
os.makedirs('data/02_interim', exist_ok=True)
os.makedirs('data/03_processed', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('reports', exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# %%
# Utility: mock CMAPSS-like dataset generator (if real data is absent)
def generate_mock_cmaps_data(n_units=100, max_cycles=200, num_sensors=21):
    """
    Produces a DataFrame similar to CMAPSS. Each unit has repeated cycles.
    We'll create smooth degradation signals for some sensors, and noise for others.
    """
    rows = []
    for unit in range(1, n_units+1):
        # sample unit-specific failure horizon to vary life (between 80 and max_cycles)
        horizon = np.random.randint(int(max_cycles*0.4), max_cycles)
        for cycle in range(1, horizon+1):
            # degrade curve: for a subset of sensors produce monotonic drift
            base = cycle / horizon
            sensors = []
            for s in range(num_sensors):
                if s % 3 == 0:
                    val = np.clip(base + np.random.normal(0, 0.02), 0, 1.0)  # degradation
                elif s % 5 == 0:
                    val = np.clip(1 - base + np.random.normal(0, 0.04), 0, 1.0)  # opposite drift
                else:
                    val = np.random.normal(0.5, 0.1)  # noisy sensor
                sensors.append(val)
            rows.append([unit, cycle] + sensors)
    cols = ['unit_id', 'cycle'] + [f'sensor_{i+1}' for i in range(num_sensors)]
    df = pd.DataFrame(rows, columns=cols)
    return df

# If no raw file present, create mock
raw_path = 'data/01_raw/train_FD_mock.txt'
if not os.path.exists(raw_path):
    print("Generating mock CMAPSS-like dataset...")
    df_mock = generate_mock_cmaps_data(n_units=120, max_cycles=250, num_sensors=21)
    df_mock.to_csv(raw_path, index=False)
    print(f"Saved mock data to {raw_path}")

# %%
# Data preprocessing and sequence creation (adapted from your script, but returns 3D arrays)
def load_and_preprocess_for_pipeline(raw_path='data/01_raw/train_FD_mock.txt', sequence_length=SEQUENCE_LENGTH):
    df = pd.read_csv(raw_path)
    # select sensor columns similar to your earlier selection
    # We'll pick 14 sensors indices (1-indexed -> pick sensors 2,3,4,7,8,9,11,12,13,15,17,20,21,14 as earlier)
    sensor_indices = [2,3,4,7,8,9,11,12,13,15,17,20,21,14]  # 1-based names
    sensor_cols = [f'sensor_{i}' for i in sensor_indices]
    # ensure columns exist in the mock
    sensor_cols = [c for c in sensor_cols if c in df.columns]
    features = ['cycle'] + sensor_cols
    df_features = df[['unit_id'] + features].copy()

    # scale features per-unit or globally? Use global MinMax for consistency
    scaler = MinMaxScaler()
    df_features[sensor_cols + ['cycle']] = scaler.fit_transform(df_features[sensor_cols + ['cycle']])

    # compute max cycles per unit -> RUL
    max_cycles = df_features.groupby('unit_id')['cycle'].max()
    def compute_rul(sub):
        uc = int(sub.iloc[0]['unit_id'])
        maxc = max_cycles.loc[uc]
        # but note cycle is scaled; compute RUL on original cycle scale would be better
        # we can reconstruct unscaled cycles: but for mock this is ok — we'll derive label by position
        return None
    # Instead, label by distance from end: last 15 cycles per unit are anomalies
    sequences = []
    labels = []
    for unit, sub in df_features.groupby('unit_id'):
        sub = sub.reset_index(drop=True)
        data = sub[features].values
        # define last K cycles as failure region
        K = 15
        n = len(data)
        label_vec = np.zeros(n, dtype=int)
        label_vec[max(0, n-K):] = 1
        # create sliding windows
        for i in range(0, n - sequence_length):
            seq = data[i:i+sequence_length]  # shape (seq_len, features)
            lbl = label_vec[i+sequence_length-1]
            sequences.append(seq)
            labels.append(lbl)
    X = np.stack(sequences, axis=0)   # (N, seq_len, num_features)
    y = np.array(labels, dtype=int)
    print(f"Created sequences: X shape {X.shape}, labels {np.bincount(y)}")
    return X, y, scaler

# Run preprocessing
X_all, y_all, scaler = load_and_preprocess_for_pipeline(raw_path)

# save initial arrays in case user wants them
np.save('data/02_interim/X_sequences.npy', X_all)
np.save('data/02_interim/y_sequences.npy', y_all)

# %%
# Phase 1: Create DataLoader of NORMAL sequences only and train VAE
normal_mask = (y_all == 0)
X_normal = X_all[normal_mask]
print(f"Normal sequences for VAE training: {X_normal.shape[0]}")

# Build DataLoader (model expects torch.Tensor batches with shape (batch, seq_len, num_features))
tensor_norm = torch.from_numpy(X_normal).float()
normal_loader = DataLoader(TensorDataset(tensor_norm), batch_size=128, shuffle=True)

# Train VAE (this calls your src.train_vae_from_dataloader implementation)
vae_model, history = train_vae_from_dataloader(normal_loader, epochs=40, lr=1e-3, kld_weight=1.0,
                                              kl_anneal={'start':0.0, 'end':1.0, 'n_epochs':20},
                                              model_savepath='models/vae_normal_manifold_weights.pth',
                                              device=DEVICE)

# Save history plot
plt.figure(figsize=(8,4))
plt.plot(history['total_loss'], label='total')
plt.plot(history['recon_loss'], label='recon')
plt.plot(history['kld_loss'], label='kld')
plt.legend(); plt.title('VAE training losses'); plt.xlabel('epoch')
plt.savefig('reports/vae_training_loss.png', dpi=150); plt.close()

# %%
# Phase 2a: Synthesis by interpolation (existing method)
# Prepare X_all and y_all as required by synthesize_and_save (X_all shape should be (N, seq_len, num_features))
synthesize_and_save(vae_model, X_all, y_all, save_dir='data/03_processed', n_per_anomaly=30)

# After synthesize_and_save: X_train_combined.npy saved (flattened)
X_combined_flat = np.load('data/03_processed/X_train_combined.npy')
y_combined = np.load('data/03_processed/y_train_combined.npy')
print("Combined flattened shapes:", X_combined_flat.shape, y_combined.shape)

# Optionally reshape to sequences for classifier
if X_combined_flat.ndim == 2:
    X_combined = X_combined_flat.reshape(X_combined_flat.shape[0], SEQUENCE_LENGTH, NUM_FEATURES)
else:
    X_combined = X_combined_flat
print("Combined reshaped:", X_combined.shape, np.bincount(y_combined.astype(int)))

# %%
# Phase 2b: Additional SMOTE-like latent augmentation
def smote_latent_perturbations(model, X_real_anomalies, normal_latent_center, n_augment_per_anom=20, sigma=0.1):
    """
    For each real anomaly latent, sample Gaussian perturbations around it (or around the midpoint to normal center)
    to create more realistic variations:
      z_base = anomaly_latent
      z_aug = z_base + N(0, sigma^2)
    Optionally, can sample around (z_base + normal_center)/2 for smoother morphs.
    Returns decoded sequences as (M * n_aug, seq_len, num_features)
    """
    # encode anomaly latents
    anomaly_latents = encode_dataset_to_latents(model, torch.from_numpy(X_real_anomalies).float().to(DEVICE), batch_size=128, device=DEVICE)
    all_aug_latents = []
    for z in anomaly_latents:
        for _ in range(n_augment_per_anom):
            # sample around z with gaussian noise (scale sigma)
            z_new = z + np.random.normal(0, sigma, size=z.shape)
            all_aug_latents.append(z_new)
    all_aug = np.stack(all_aug_latents, axis=0)
    # decode
    decoded = decode_latents_to_sequences(model, all_aug, batch_size=128, device=DEVICE)
    return decoded

# prepare real anomaly sequences (3D)
anomaly_mask = (y_all == 1)
X_real_anoms = X_all[anomaly_mask]
print("Real anomalies:", X_real_anoms.shape[0])

# compute normal latent center
normal_tensor = torch.from_numpy(X_all[normal_mask]).float().to(DEVICE)
normal_latents = encode_dataset_to_latents(vae_model, normal_tensor, batch_size=128, device=DEVICE)
normal_center = np.mean(normal_latents, axis=0)

# generate SMOTE-like augmentations
synthetic_smote = smote_latent_perturbations(vae_model, X_real_anoms, normal_center, n_augment_per_anom=25, sigma=0.08)
print("SMOTE-like synthetic shape:", synthetic_smote.shape)

# flatten and append to combined
synth_smote_flat = synthetic_smote.reshape(synthetic_smote.shape[0], -1)
X_combined_flat2 = np.concatenate([X_combined_flat, synth_smote_flat], axis=0)
y_synth_smote = np.ones(synth_smote_flat.shape[0], dtype=int)
y_combined2 = np.concatenate([y_combined, y_synth_smote], axis=0)
print("New combined shapes:", X_combined_flat2.shape, y_combined2.shape)

# Save extended combined dataset
np.save('data/03_processed/X_train_combined_augmented.npy', X_combined_flat2)
np.save('data/03_processed/y_train_combined_augmented.npy', y_combined2)

# %%
# Phase 3: Train LSTM classifier on augmented data
# We'll implement training similar to the train_classifier.py logic (but inline here)
# Load augmented dataset
X_flat = np.load('data/03_processed/X_train_combined_augmented.npy')
y = np.load('data/03_processed/y_train_combined_augmented.npy')

# reshape to (N, seq_len, num_features)
X_seq = X_flat.reshape(-1, SEQUENCE_LENGTH, NUM_FEATURES)

# Stratified split for train/val
X_train, X_val, y_train, y_val = train_test_split(X_seq, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y)

# Dataloaders
train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long()),
                          batch_size=128, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long()),
                        batch_size=128, shuffle=False)

# LSTM classifier (small model)
class LSTMClassifier(nn.Module):
    def __init__(self, num_features=NUM_FEATURES, hidden_size=128, num_layers=1, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2)
        )
    def forward(self, x):
        out, (h_n, _) = self.lstm(x)
        last = h_n[-1]
        return self.fc(last)

# Instantiate model
clf = LSTMClassifier().to(DEVICE)

# class weights
classes, counts = np.unique(y_train, return_counts=True)
inv_freq = 1.0 / counts
norm = inv_freq / np.sum(inv_freq) * len(classes)
weights = np.zeros(len(classes), dtype=np.float32)
for c,w in zip(classes, norm):
    weights[c] = w
class_weights = torch.tensor(weights, dtype=torch.float32).to(DEVICE)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(clf.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=4, verbose=True)

best_f1 = -1.0
best_state = None
history_clf = {'epoch':[], 'loss':[], 'val_p':[], 'val_r':[], 'val_f1':[]}
for epoch in range(1, 31):
    clf.train()
    running_loss = 0.0
    for Xb, yb in train_loader:
        Xb = Xb.to(DEVICE); yb = yb.to(DEVICE)
        optimizer.zero_grad()
        logits = clf(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * Xb.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)

    # validation
    clf.eval()
    all_preds = []; all_labels = []
    with torch.no_grad():
        for Xv, yv in val_loader:
            Xv = Xv.to(DEVICE)
            logits = clf(Xv)
            preds = torch.softmax(logits, dim=1)[:,1].cpu().numpy()  # anomaly probability
            all_preds.append(preds)
            all_labels.append(yv.numpy())
    y_true = np.concatenate(all_labels)
    y_score = np.concatenate(all_preds)
    # default threshold 0.5
    y_pred_bin = (y_score >= 0.5).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred_bin, labels=[1], average='binary', zero_division=0)
    print(f"Epoch {epoch} | Loss {epoch_loss:.5f} | Val P {p:.4f} R {r:.4f} F1 {f1:.4f}")
    history_clf['epoch'].append(epoch); history_clf['loss'].append(epoch_loss)
    history_clf['val_p'].append(p); history_clf['val_r'].append(r); history_clf['val_f1'].append(f1)

    scheduler.step(f1)
    if f1 > best_f1:
        best_f1 = f1
        best_state = clf.state_dict().copy()
        torch.save(best_state, 'models/classifier_best_by_f1.pth')
        # compute confusion matrix at 0.5 for record
        cm = confusion_matrix(y_true, y_pred_bin)
        plt.figure(figsize=(4,3)); sns.heatmap(cm, annot=True, fmt='d'); plt.title(f'Best CM Epoch {epoch}'); plt.savefig('reports/confusion_matrix_best.png'); plt.close()
        print(f"Saved best classifier at epoch {epoch} with F1 {f1:.4f}")

# save training history
pd.DataFrame(history_clf).to_csv('reports/classifier_history.csv', index=False)

# %%
# Phase 4: Threshold calibration on validation set (precision-recall curve)
# load best model
clf.load_state_dict(best_state)
clf.to(DEVICE)
clf.eval()

# get validation scores
all_scores = []; all_y = []
with torch.no_grad():
    for Xv, yv in val_loader:
        Xv = Xv.to(DEVICE)
        logits = clf(Xv)
        probs = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        all_scores.append(probs); all_y.append(yv.numpy())
y_val_true = np.concatenate(all_y)
y_val_scores = np.concatenate(all_scores)

# get precision-recall
precision, recall, thresholds = precision_recall_curve(y_val_true, y_val_scores)
f1s = (2 * precision * recall) / (precision + recall + 1e-12)
best_idx = np.argmax(f1s)
best_threshold = 0.5 if thresholds.size == 0 else (thresholds[best_idx] if best_idx < len(thresholds) else 0.5)
best_prec = precision[best_idx]; best_rec = recall[best_idx]; best_f1_thr = f1s[best_idx]
print(f"Best threshold by val F1: {best_threshold:.4f} (P={best_prec:.4f}, R={best_rec:.4f}, F1={best_f1_thr:.4f})")

# Plot precision-recall curve and mark chosen threshold
plt.figure(figsize=(6,4))
plt.plot(recall, precision, label='Precision-Recall curve')
plt.scatter([best_rec], [best_prec], c='red', label=f'Best F1 @thr={best_threshold:.3f}')
plt.xlabel('Recall'); plt.ylabel('Precision'); plt.legend()
plt.title('Precision-Recall Curve (Validation)')
plt.savefig('reports/precision_recall_curve.png'); plt.close()

# Save the chosen threshold to disk for inference
with open('models/threshold.txt', 'w') as f:
    f.write(str(float(best_threshold)))
print("Saved threshold to models/threshold.txt")

# %%
# Fi
